In [1]:
from PIL import Image




In [2]:
# Insert you nation's tag (used for convenience in naming)
tag = 'TEST'

In [3]:
flag_name = tag + '_Flag.png'
battle_flag_name = 'BattleFlag_' + tag + '.png'
right_name = tag + '_Flag_Wave_Right.png'
left_name = tag + '_Flag_Wave_Left.png'
camp_name = tag + '_Camp.png'

In [4]:
image1 = Image.open('image_source.png')

In [5]:
# Resizes image for correct resolution
resized_image = image1.resize((256, 256))

# Saves flag file
resized_image.save(flag_name)


In [6]:
# Downside of Battle flag (512x70)
image2 = Image.open('downside.png')  

# New image for battle flag 512x512
final_image = Image.new('RGBA', (512, 512), (255, 255, 255, 0))  # Transparent background

# Resize image for battle flag
resized_image1 = image1.resize((512, 512 - 70))

# Combine
final_image.paste(resized_image1, (0, 0))  

final_image.paste(image2, (0, 512-70)) 

# Save Battle flag
final_image.save(battle_flag_name)

In [7]:
def shift_pixels(image_path, shifted_image_path, shift=20):
    # Function which shifts image for wavey flags
    image = Image.open(image_path).convert("RGBA")
    width, height = image.size
    
    # file for result
    new_image = Image.new("RGBA", (width, height + shift))
    
    # Cycle for every pixel
    for x in range(width):
        for y in range(height):
            
            new_y = y + (x / (width / shift))
            
            if new_y >= height + shift:
                new_y = height + shift - 1
            
            pixel_color = image.getpixel((x, y))
            
            new_image.putpixel((x, int(new_y)), pixel_color)

    # saves shifted image
    new_image.save(shifted_image_path)

# 
image_path = flag_name  
shifted_image_path = 'shifted_image.png'  

# Shift image
shift_pixels(image_path, shifted_image_path, shift=20)

In [8]:
def overlay_hanging_flag_on_texture(texture_image_path, hanging_flag_image_path, output_image_path, alpha=0.5):
    # Function creates wavey flags
    texture_image = Image.open(texture_image_path).convert("RGBA")
    
    # Hanging flag mask
    hanging_flag_image = Image.open(hanging_flag_image_path).convert("RGBA")
    
    # 
    bbox = hanging_flag_image.getbbox()
    
    # О
    if bbox:
        hanging_flag_image = hanging_flag_image.crop(bbox)

    # Resize hanging flag yp fit texture
    hanging_flag_image = hanging_flag_image.resize(texture_image.size)

    # New image for result
    combined_image = Image.new("RGBA", texture_image.size)

    # Combine
    combined_image = Image.alpha_composite(combined_image, texture_image)

    # 
    hanging_flag_with_alpha = Image.new("RGBA", hanging_flag_image.size)
    for x in range(hanging_flag_image.width):
        for y in range(hanging_flag_image.height):
            r, g, b, a = hanging_flag_image.getpixel((x, y))
            # 
            new_alpha = int(a * alpha)
            hanging_flag_with_alpha.putpixel((x, y), (r, g, b, new_alpha))

    # 
    mask = hanging_flag_with_alpha.split()[3]  # 

    # 
    combined_image = Image.alpha_composite(combined_image, hanging_flag_with_alpha)

    # 
    for x in range(hanging_flag_with_alpha.width):
        for y in range(hanging_flag_with_alpha.height):
            _, _, _, a = hanging_flag_with_alpha.getpixel((x, y))
            if a == 0:  # 
                combined_image.putpixel((x, y), (0, 0, 0, 0))  # Устанавливаем пиксель в прозрачный

    # save for camp flag
    combined_image.save("temp.png")
    
    # Turn image 27 degree 
    combined_image = combined_image.rotate(-27, expand=True)

    # Resize
    combined_image = combined_image.resize((254, 316))

    # Save
    combined_image.save(output_image_path)

# Paths
texture_image_path = 'shifted_image.png'  
hanging_flag_image_path = 'hanging_flag.png'  
output_image_path = 'combined_flag.png'  

# Combine
overlay_hanging_flag_on_texture(texture_image_path, hanging_flag_image_path, output_image_path, alpha=0.5)

combined_flag = Image.open(output_image_path)
# Mirror
combined_flag2 = combined_flag.transpose(Image.FLIP_LEFT_RIGHT)

In [9]:
combined_flag.save(right_name)
combined_flag2.save(left_name)

In [10]:
temp_file = Image.open("temp.png")

# New image for camp flag
temp_image = Image.new("RGBA", (300, 300), (0, 0, 0, 0))

# 
x_offset = (temp_file.width - temp_file.width) // 2
y_offset = (temp_file.height - temp_file.height) // 2

# 
temp_image.paste(temp_file, (x_offset, y_offset), temp_file)


In [11]:
def create_image_grid(images, grid_width, grid_height, image_size, output_image_path):
    # 
    grid_image = Image.new("RGBA", (grid_width * image_size[0], grid_height * image_size[1]), (0, 0, 0, 0))

    # 
    for index, img in enumerate(images):
        x = (index % grid_width) * image_size[0]
        y = (index // grid_width) * image_size[1]
        grid_image.paste(img, (x, y))

    # е
    grid_image.save(output_image_path)

output_image_path = camp_name  

# 
images = [temp_image] * (8 * 6)  #

# Creates camp flag
create_image_grid(images, grid_width=8, grid_height=6, image_size=(300, 300), output_image_path=output_image_path)
